# Part One

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob
import copy 

## Pulling Info From YML File

In [2]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [3]:
config_filename

'./ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
first_ID = config['inputs']['first_ID']

In [6]:
second_ID = config['inputs']['second_ID']

In [7]:
output_path = config['inputs']['output_path']

In [8]:
first_ID_datafiles = config[first_ID].values()

In [9]:
second_ID_datafiles = config[second_ID].values()

## Downloading SQL Data

In [10]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [11]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [12]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [13]:
# SQl Data at different levels 
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction']
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa']

## Downloading the first_ID Data

In [45]:
# first_ID Data Frame
first_ID_df = pd.DataFrame()
for file_name in first_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    first_ID_df = first_ID_df.append(working_df)

In [16]:
# Save the features_first_ID for future use (Used when creating the diff file)
features_first_ID = first_ID_df.drop(['mgra', 'year'], axis=1).columns

In [51]:
comparison_first_ID_no_geozone = copy.deepcopy(first_ID_df)

In [52]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data = comparison_first_ID_no_geozone.merge(jur_level, how='left', on='mgra')

In [54]:
# Drop redundant geotype column
comparison_first_ID_processed_data = comparison_first_ID_processed_data.drop('geotype', axis=1)

In [56]:
# making it original
comparison_first_ID_processed_data.columns = [x + f'_{first_ID}' for x in comparison_first_ID_processed_data.columns]

In [79]:
comparison_first_ID_processed_data.head()

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,hotelroomtotal_DS35,luz_id_DS35,truckregiontype_DS35,district27_DS35,milestocoast_DS35,acres_DS35,effective_acres_DS35,land_acres_DS35,year_DS35,geozone_DS35
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego


In [22]:
#comparison_first_ID_processed_data.to_csv(f"{output_path}/comparison_{first_ID}_processed_data.csv")

## Downloading  the second_ID Data

In [48]:
# second_id Data Frame
second_ID_df = pd.DataFrame()
for file_name in second_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    second_ID_df = second_ID_df.append(working_df)

In [49]:
# Save the features of the second ID for future use (Used when creating the diff file)
features_second_ID = second_ID_df.drop(['mgra', 'year'], axis=1).columns

In [58]:
comparison_second_ID_no_geozone = copy.deepcopy(second_ID_df)

In [60]:
# Adding SQl Data to second_id_df
comparison_second_ID_processed_data = comparison_second_ID_no_geozone.merge(jur_level, how='left', on='mgra')

In [61]:
# Drop redundant geotype column
comparison_second_ID_processed_data = comparison_second_ID_processed_data.drop('geotype', axis=1)

In [63]:
# making it original
comparison_second_ID_processed_data.columns = [x + f'_{second_ID}' for x in comparison_second_ID_processed_data.columns]

In [80]:
comparison_second_ID_processed_data.head()

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41,geozone_DS41
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego


In [31]:
# comparison_second_ID_processed_data.to_csv(f"{output_path}/comparison_{second_ID}_processed_data.csv")

## Concatenate both DS dataframes

In [70]:
# Inner join dataframes on mgra and year
first_second_ID_comparison = first_ID_df.merge(second_ID_df, on=["mgra", "year"], suffixes=[f'_{first_ID}', f'_{second_ID}'])

In [73]:
# Aggregate sum by mgra and year values
first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()

In [81]:
first_second_ID_comparison.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra year                                                                   
1    2016      3331       19          19           0           0       18   
     2018      3331       19          19           0           0       18   
     2020      3331       19          19           0           0       18   
     2023      3331       20          20           0           0       18   
     2025      3331       20          20           0           0       18   

           hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra year                                                   ...   
1    2016          18           0           0            0  ...   
     2018          18           0           0            0  ...   
     2020          18           0           0            0  ...   
     2023          18           0           0            0  ...   
     2025          18           0           0            0  ...   

           midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra year                                                             
1    2016                  0                 0                    0   
     2018                  0                 0                    0   
     2020                  0                 0                    0   
     2023                  0                 0                    0   
     2025                  0                 0                    0   

           luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra year                                                       
1    2016           95                     1               27   
     2018           95                     1               27   
     2020           95                     1               27   
     2023           95                     1               27   
     2025           95                     1               27   

           milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra year                                                        
1    2016             3.7997   16.615444             12.961482   
     2018             3.7997   16.615444             12.961482   
     2020             3.7997   16.615444             12.961482   
     2023             3.7997   16.615444             12.961482   
     2025             3.7997   16.615444             12.961482   

           land_acres_DS41  
mgra year                   
1    2016        16.615444  
     2018        16.615444  
     2020        16.615444  
     2023        16.615444  
     2025        16.615444  

[5 rows x 206 columns]

In [ ]:
#first_second_ID_comparison.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison")

# Creating the complete Diff File

In [75]:
# Finding features common to both DSID data frames
list1_as_set = set(features_first_ID)
intersection = list1_as_set.intersection(features_second_ID)

shared_features = list(intersection)

In [76]:
# Calculate diff values between the two DS_ID's
diff_df = pd.DataFrame()
for column in shared_features:
    diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']

In [78]:
diff_df.head()

taz_diff  hotelroomtotal_diff  beachactive_diff  emp_retail_diff  \
mgra year                                                                     
1    2016         0                    0               0.0                0   
     2018         0                    0               0.0                0   
     2020         0                    0               0.0                0   
     2023         0                    0               0.0                0   
     2025         0                    0               0.0                0   

           hs_mh_diff  emp_total_diff  hh_mh_diff  numfreehrs_diff  hhp_diff  \
mgra year                                                                      
1    2016           0               0           0                0         0   
     2018           0               0           0                0         3   
     2020           0               0           0                0         6   
     2023           0               0           0                0         6   
     2025           0               0           0                0        -2   

           i9_diff  ...  dparkcost_diff  emp_personal_svcs_retail_diff  \
mgra year           ...                                                  
1    2016        0  ...               0                              0   
     2018        1  ...               0                              0   
     2020       -1  ...               0                              0   
     2023        0  ...               0                              0   
     2025        2  ...               0                              0   

           hparkcost_diff  emp_health_diff  emp_state_local_gov_white_diff  \
mgra year                                                                    
1    2016               0                0                               0   
     2018               0                0                               0   
     2020               0                0                               0   
     2023               0                0                               0   
     2025               0                0                               0   

           popden_diff  dudenbin_diff  i2_diff  emp_pvt_ed_post_k12_oth_diff  \
mgra year                                                                      
1    2016          0.0              0        0                             0   
     2018          0.0              0       -1                             0   
     2020          0.0              0       -2                             0   
     2023          0.0              0        0                             0   
     2025          0.0              0       -1                             0   

           emp_pvt_hh_diff  
mgra year                   
1    2016                0  
     2018                0  
     2020                0  
     2023                0  
     2025                0  

[5 rows x 103 columns]

In [82]:
#diff_df.to_csv(f"{output_path}/{first_ID}_{second_ID}_complete_diff")